In [2]:
from datetime import datetime
import numpy as np
import pandas as pd

# from utilities import *
import opendp.prelude as dp


In [3]:
dp.enable_features("contrib", "floating-point", "honest-but-curious")

# PUBLIC INFO
start_date, end_date = datetime(2020, 9, 1), datetime(2021, 3, 31)
time_col = "date"

# DATA
path = "C:\\Users\kshub\\OneDrive\\Documents\\PET_phase_2\\Technical_Phase_Data\\technical_phase_data.csv"
df = pd.read_csv(path)

<>:8: SyntaxWarning: invalid escape sequence '\k'
<>:8: SyntaxWarning: invalid escape sequence '\k'
C:\Users\kshub\AppData\Local\Temp\ipykernel_22960\3898676667.py:8: SyntaxWarning: invalid escape sequence '\k'
  path = "C:\\Users\kshub\\OneDrive\\Documents\\PET_phase_2\\Technical_Phase_Data\\technical_phase_data.csv"


In [129]:
def dataframe_domain(public_key_sets=None):
    """Creates a domain representing the set of all data frames.
    
    Assumes column names and types are public information.
    Key sets optionally named for columns in `public_key_sets` are considered public information.

    Two data frames differing in their public information 
    are considered to have a data set distance of infinity.
    """
    return dp.user_domain(
        "DataFrameDomain", lambda x: isinstance(x, pd.DataFrame), public_key_sets
    )


def series_domain():
    """Creates a domain representing the set of all series.

    Assumes series name and type are public information.

    Two series differing in their public information 
    are considered to have a data set distance of infinity.
    """
    return dp.user_domain("SeriesDomain", lambda x: isinstance(x, pd.Series))

def identifier_distance():
    """Symmetric distance between the id sets."""
    return dp.user_distance("IdentifierDistance")


def approx_concentrated_divergence():
    """symmetric distance between the id sets"""
    return dp.user_distance("ApproxConcentratedDivergence()")


In [169]:
def make_preprocess_location():
    """Create a 1-stable transformation to bin `merch_postal_code` by city"""

    def categorize_city(code):
        if code.startswith("5"):
            return "Medellin"
        elif code.startswith("11"):
            return "Bogota"
        elif code.startswith("70"):
            return "Brasilia"
        else:
            return "Santiago"

    def location_preprocess(df):
        loc_df = df.copy()
        # Convert merchant_postal_code into str type
        loc_df["merch_postal_code"] = loc_df["merch_postal_code"].astype(str)
        # Apply the function to create a new column
        loc_df["city"] = loc_df["merch_postal_code"].apply(
            categorize_city
        )
        return loc_df

    return dp.t.make_user_transformation(
        input_domain=dataframe_domain(),
        input_metric=identifier_distance(),
        output_domain=dataframe_domain(),
        output_metric=identifier_distance(),
        function=location_preprocess,
        stability_map=lambda d_in: d_in,
    )

In [183]:
def make_preprocess_merchant():
    """Create a 1-stable transformation to bin `merch_postal_code` by city"""

    def categorize_merchant(merch):
        if merch in ['Hotels/Motels','Restaurants','Bars/Discotheques']:
            return "luxury"
        elif merch in ['Grocery Stores/Supermarkets','Drug Stores/Pharmacies','General Retail Stores','Utilities: Electric, Gas, Water','Hospitals']:
            return "essential"
        else:
            return "other"

    def merchant_preprocess(df):
        loc_df = df.copy()
        # Convert merchant_postal_code into str type
        loc_df["merch_category"] = loc_df["merch_category"].astype(str)
        # Apply the function to create a new column
        loc_df["merch_super_category"] = loc_df["merch_category"].apply(
            categorize_merchant
        )
        return loc_df

    return dp.t.make_user_transformation(
        input_domain=dataframe_domain(),
        input_metric=identifier_distance(),
        output_domain=dataframe_domain(),
        output_metric=identifier_distance(),
        function=merchant_preprocess,
        stability_map=lambda d_in: d_in,
    )

In [170]:
def make_truncate_time(start_date, end_date, time_col):
    """Create a transformation that filters the data to a given time frame.
    
    WARNING: Assumes that the data has at most one contribution per individual per week.
    """
    number_of_timesteps = (end_date - start_date).days // 7

    def time_preprocess(df):
        df = df.copy()

        # Convert time_col into datetime type
        df[time_col] = pd.to_datetime(df[time_col])

        # Filter the DataFrame based on the specified dates
        return df[(df[time_col] >= start_date) & (df[time_col] <= end_date)]

    return dp.t.make_user_transformation(
        input_domain=dataframe_domain(),
        input_metric=identifier_distance(),
        output_domain=dataframe_domain(),
        output_metric=dp.symmetric_distance(),
        function=time_preprocess,
        stability_map=lambda d_in: d_in * number_of_timesteps,
    )

In [171]:
def make_sum_by(column, by, bounds):
    """Create a transformation that computes the grouped bounded sum of `column`"""
    L, U = bounds
    # print(df.head())
    def function(df):
        df = df.copy()
        # print(df.head())
        df[column] = df[column].clip(*bounds)
        return df.groupby(by)[column].sum()

    return dp.t.make_user_transformation(
        input_domain=dataframe_domain(),
        input_metric=dp.symmetric_distance(),
        output_domain=series_domain(),
        output_metric=dp.l2_distance(T=float),
        function=function,
        stability_map=lambda d_in: np.sqrt(d_in) * max(abs(L), U),
    )


maximum nb_transaction entry for any category is 454. Assuming the bound of [0, 454]

In [172]:

def make_private_sum_by(column, by, bounds, scale):
    """Create a measurement that computes the grouped bounded sum of `column`"""
    space = dp.vector_domain(dp.atom_domain(T=int)), dp.l2_distance(T=float)
    m_gauss = space >> dp.m.then_gaussian(scale)
    t_sum = make_sum_by(column, by, bounds)

    def function(df):
        exact = t_sum(df)
        # print(exact)
        # print(exact.to_numpy())
        noisy_sum = pd.Series(
            np.maximum(m_gauss(exact.to_numpy().flatten()), 0), index=exact.index
        )
        return noisy_sum.to_frame()

    return dp.m.make_user_measurement(
        input_domain=dataframe_domain(public_key_sets=[by]),
        input_metric=dp.symmetric_distance(),
        output_measure=dp.zero_concentrated_divergence(T=float),
        function=function,
        privacy_map=lambda d_in: m_gauss.map(t_sum.map(d_in)),
    )

In [173]:
def make_filter(column,entry, sensetivity:int= 1):
        """filters offline entries"""
        
        def function(df):
            df = df.copy()
            return df[(df[column] == entry)]


        return dp.t.make_user_transformation(
        input_domain=dataframe_domain(),
        input_metric=identifier_distance(),
        output_domain=dataframe_domain(),
        output_metric=identifier_distance(),
        function=function,
        stability_map=lambda d_in: d_in* sensetivity,
    )

In [174]:
df_new = df.copy()
bounds = (0, 454)
start_date, end_date = datetime(2020, 9, 1), datetime(2021, 3, 31)
columns = "nb_transactions"
by = "merch_postal_code"
scale=10.0
column="transaction_type"
entry="OFFLINE"
city_col="city"
City_entry="Medellin"
hotspot_predictor=(
    make_preprocess_location()
    >>make_filter(column,entry)
    >>make_filter(city_col,City_entry)
    >>make_truncate_time(start_date, end_date, time_col)
    >>make_private_sum_by(columns, by, bounds, scale)
)
print(hotspot_predictor.map(1))
output=hotspot_predictor(df_new)
print(output)

30917.40000000001
                        0
merch_postal_code        
500001             182278
500002             184203
500003             181031
500004             178535
500005             202209
500006             189736
500007             205075
500008             166502
500009             194875
500010             188914
500011             175243
500012             165526
500013             190525
500014             214841
500015             191984
500016             192572
500017             193764
500020             218498
500021             205332
500022             214301
500023             185691
500024             212903
500025             206197
500026             185104
500027             202576
500028             180739
500030             207620
500031             207334
500032             221125
500033             193689
500034             176485
500035             190021
500036             176725
500037             163625
500040             178749
500041             1

In [175]:
def hotspot_analyser(df:pd.DataFrame, start_date:datetime,end_date:datetime,city_filter:str, nb_postal_codes: int,epsilon:float):
    """final function to predict hotspots"""
    bounds = (0, 600)
    transaction_data_col = "nb_transactions"
    postal_code_groupby_col = "merch_postal_code"
    transaction_type_col = "transaction_type"
    transaction_type_filter = "OFFLINE"
    city_col="city"
    time_col="date"

    """time steps calculation"""
    nb_timesteps = (end_date - start_date).days // 7

    """scale calculation"""
    scale=(3.0*nb_postal_codes*nb_timesteps)/epsilon

    new_df=df.copy()


    hotspot_predictor=(
    make_preprocess_location()
    >>make_filter(transaction_type_col,transaction_type_filter)
    >>make_filter(city_col,city_filter,nb_postal_codes)
    >>make_truncate_time(start_date, end_date, time_col)
    >>make_private_sum_by(transaction_data_col, postal_code_groupby_col, bounds, scale)
   )

    return hotspot_predictor(new_df)
    

In [178]:
print(hotspot_analyser(df,start_date,end_date,"Medellin",42,10))

                        0
merch_postal_code        
500001             182127
500002             184834
500003             180723
500004             178663
500005             202160
500006             190032
500007             204808
500008             166404
500009             194555
500010             189101
500011             175437
500012             165264
500013             190181
500014             214967
500015             191939
500016             193132
500017             194186
500020             218432
500021             205058
500022             215002
500023             186325
500024             213090
500025             206436
500026             185552
500027             203101
500028             180149
500030             207686
500031             207353
500032             221942
500033             193726
500034             177103
500035             189928
500036             176520
500037             163407
500040             178941
500041             185563
500042      

In [179]:
def mobility_analyser(df:pd.DataFrame,start_date:datetime,end_date:datetime,city_filter: str, epsilon:float):
    """final function to predict hotspots"""
    bounds = (0, 600)
    transaction_data_col = "nb_transactions"
    groupby_col = "date"

    city_col="city"
    time_col="date"
    merch_category_col="merch_category"
    merch_filter="Airlines"

    """time steps calculation"""
    nb_timesteps = (end_date - start_date).days // 7

    """scale calculation"""
    scale=(3.0*nb_timesteps*nb_timesteps)/epsilon

    new_df=df.copy()


    hotspot_predictor=(
    make_preprocess_location()
    >>make_filter(city_col,city_filter)
    >>make_filter(merch_category_col,merch_filter)
    >>make_truncate_time(start_date, end_date, time_col)
    >>make_private_sum_by(transaction_data_col, groupby_col, bounds, scale)
   )

    return hotspot_predictor(new_df)

In [182]:
print(mobility_analyser(df,start_date,end_date,"Medellin",10))

               0
date            
2020-09-01  1418
2020-09-08   860
2020-09-15  1273
2020-09-22  1278
2020-09-29   937
2020-10-06  1413
2020-10-13  1185
2020-10-20  1055
2020-10-27   981
2020-11-03  1431
2020-11-10  1260
2020-11-17  1270
2020-11-24  1736
2020-12-01  1331
2020-12-08  1081
2020-12-15  1601
2020-12-22  1591
2020-12-29  1300
2021-01-05  1082
2021-01-12  1873
2021-01-19  1244
2021-01-26  1223
2021-02-02  1560
2021-02-09  1747
2021-02-16  1267
2021-02-23  2057
2021-03-02  1117
2021-03-09  1147
2021-03-16  1608
2021-03-23  1488
2021-03-30   945


In [185]:
def pandemic_stage_analyser(df:pd.DataFrame,start_date:datetime,end_date:datetime,city_filter: str,essential_or_luxury:str, epsilon:float):
    """final function to predict hotspots"""
    bounds = (0, 600)
    transaction_data_col = "nb_transactions"
    groupby_col = "date"

    city_col="city"
    time_col="date"
    merch_category_col="merch_super_category"


    """time steps calculation"""
    nb_timesteps = (end_date - start_date).days // 7

    """scale calculation"""
    scale=(3.0*nb_timesteps*nb_timesteps)/epsilon

    new_df=df.copy()


    hotspot_predictor=(
    make_preprocess_location()
    >>make_preprocess_merchant()
    >>make_filter(city_col,city_filter)
    >>make_filter(merch_category_col,essential_or_luxury)
    >>make_truncate_time(start_date, end_date, time_col)
    >>make_private_sum_by(transaction_data_col, groupby_col, bounds, scale)
   )

    return hotspot_predictor(new_df)

In [187]:
print(pandemic_stage_analyser(df,start_date,end_date,"Medellin",essential_or_luxrious="luxury",epsilon=10))

                 0
date              
2020-09-01  119525
2020-09-08  122797
2020-09-15  117481
2020-09-22  123151
2020-09-29  115222
2020-10-06  118614
2020-10-13  117163
2020-10-20  117853
2020-10-27  116842
2020-11-03  117436
2020-11-10  124451
2020-11-17  115435
2020-11-24  117502
2020-12-01  117037
2020-12-08  110991
2020-12-15  108300
2020-12-22  114715
2020-12-29  107363
2021-01-05  106208
2021-01-12  105092
2021-01-19  109456
2021-01-26  115614
2021-02-02  122143
2021-02-09  125693
2021-02-16  127317
2021-02-23  131396
2021-03-02  133496
2021-03-09  126942
2021-03-16  122216
2021-03-23  118228
2021-03-30  112776


In [157]:
scale=dp.binary_search_param(hotspot_predictor, 1,1.0,bounds=bounds)
print(scale)

OpenDPException: 
  FFI("Continued stack trace from Exception in user-defined function:
Traceback (most recent call last):
  File "c:\Users\kshub\AppData\Local\Programs\Python\Python312\Lib\site-packages\opendp\_convert.py", line 459, in wrapper_func
    py_out = func(py_arg)
             ^^^^^^^^^^^^
  File "C:\Users\kshub\AppData\Local\Temp\ipykernel_22960\4272251858.py", line 5, in function
    df = df.copy()
         ^^^^^^^
AttributeError: 'int' object has no attribute 'copy'
")

In [122]:
analysis=dp.binary_search_chain(hotspot_predictor, 1,1.0,bounds=bounds)

OpenDPException: 
  FFI("Continued stack trace from Exception in user-defined function:
Traceback (most recent call last):
  File "c:\Users\kshub\AppData\Local\Programs\Python\Python312\Lib\site-packages\opendp\_convert.py", line 459, in wrapper_func
    py_out = func(py_arg)
             ^^^^^^^^^^^^
  File "C:\Users\kshub\AppData\Local\Temp\ipykernel_22960\257734847.py", line 8, in <lambda>
    function=lambda df: df[(df["transaction_type"] == "OFFLINE")],
                            ~~^^^^^^^^^^^^^^^^^^^^
TypeError: 'int' object is not subscriptable
")

In [149]:
df_new = df.copy()
bounds = (0, 454)
start_date, end_date = datetime(2020, 9, 1), datetime(2021, 3, 31)
columns = ["nb_transactions"]
by = ["merch_postal_code"]
scale=10.0
column="transaction_type"
entry="OFFLINE"
hotspot_predictor=(
    make_filter_offline(column,entry)
    >>make_truncate_time(start_date, end_date, time_col)
    >>make_private_sum_by(columns, by, bounds, scale)
)
print(hotspot_predictor.map(1))
output=hotspot_predictor(df_new)
print(output)

30917.40000000001
                        0
merch_postal_code        
7071                45765
55411              183692
70040               31856
70050               27510
70070               43102
...                   ...
9670000            108717
9710000            106582
9750000             84828
9790000            128256
9810000             83009

[303 rows x 1 columns]


In [150]:
df_new = df.copy()
bounds = (0, 454)
start_date, end_date = datetime(2020, 9, 1), datetime(2021, 3, 31)
columns = ["nb_transactions"]
by = ["merch_postal_code","date"]
scale=10.0
column="merch_category"
entry="Drug Stores/Pharmacies"
hotspot_predictor=(
    make_filter_offline(column,entry)
    >>make_truncate_time(start_date, end_date, time_col)
    >>make_private_sum_by(columns, by, bounds, scale)
)
print(hotspot_predictor.map(1))
output=hotspot_predictor(df_new)
print(output)

30917.40000000001
                                 0
merch_postal_code date            
7071              2020-09-01  2014
                  2020-09-08  1946
                  2020-09-15  1923
                  2020-09-22  1940
                  2020-09-29  1896
...                            ...
9810000           2021-03-02  1836
                  2021-03-09  1851
                  2021-03-16  1899
                  2021-03-23  1907
                  2021-03-30  1923

[9238 rows x 1 columns]


In [151]:
df_new = df.copy()
bounds = (0, 454)
start_date, end_date = datetime(2020, 9, 1), datetime(2021, 3, 31)
columns = ["nb_transactions"]
by = ["merch_postal_code","date"]
scale=10.0
column="merch_category"
entry="Airlines"
hotspot_predictor=(
    make_filter_offline(column,entry)
    >>make_truncate_time(start_date, end_date, time_col)
    >>make_private_sum_by(columns, by, bounds, scale)
)
print(hotspot_predictor.map(1))
output=hotspot_predictor(df_new)
print(output)

30917.40000000001
                               0
merch_postal_code date          
7071              2020-09-01   9
                  2020-09-08  25
                  2020-09-15  18
                  2020-09-22   0
                  2020-09-29   6
...                           ..
9810000           2021-03-02  58
                  2021-03-09  59
                  2021-03-16  71
                  2021-03-23  53
                  2021-03-30  65

[6291 rows x 1 columns]


In [ ]:
def make_select_grouping_cols(candidates, min_bin_size, d_in, d_out):
    """Create a measurement that selects a set of grouping columns from `candidates`."""
    def make(s):
        return (
            make_grouping_cols_score(candidates, min_bin_size)
            >> dp.m.then_report_noisy_max_gumbel(s, optimize="max")
            >> (lambda idx: candidates[idx])
        )

    return dp.binary_search_chain(make, d_in, d_out, T=float)

In [28]:
def hotspot_detection(df, start_date, end_date, time_col):
    """DP function that detects hotspots."""
    df_new = df.copy()
    bounds = (0, 454)

    def make_filter_offline():
        """filters offline entries"""
        return dp.t.make_user_transformation(
        input_domain=dataframe_domain(),
        input_metric=dp.symmetric_distance(),
        output_domain=dp.vector_domain(dp.atom_domain(T=T)),
        output_metric=dp.symmetric_distance(),
        function=lambda df: df[(df["transaction_type"] == "OFFLINE")],
        stability_map=lambda d_in: d_in,
    )




    make_filter_offline>>make_truncate_time(start_date, end_date, time_col)>>make_private_sum_by()
    